# Bradley-Terry Ranking Tutorial: Probabilistic Model Comparison

## Learning Objectives

By the end of this tutorial, you will:
- ✅ Understand the Bradley-Terry model and its probabilistic foundation
- ✅ Learn maximum likelihood estimation (MLE) for skill parameters
- ✅ Fit Bradley-Terry model to pairwise comparison data
- ✅ Visualize skill estimates with uncertainty (standard errors)
- ✅ Compare Bradley-Terry vs Elo rankings
- ✅ Decide when to use Bradley-Terry vs Elo

## Execution Details

- **Execution Time:** <4 minutes
- **Cost:** $0 (simulation-based, no API calls)
- **Prerequisites:** Understanding of comparative evaluation and Elo ranking

## Background

The **Bradley-Terry model** (1952) is a probabilistic approach to ranking:
- Each model has a latent "skill" parameter θ
- Win probability follows logistic function
- Uses **all comparison data simultaneously** (batch estimation)
- Provides **uncertainty estimates** (standard errors)

**Differences from Elo:**
- Elo: Sequential updates (online), no uncertainty
- Bradley-Terry: Batch estimation (offline), with uncertainty

**Real-world uses:**
- Sports rankings (NFL, NBA) - one-time seasonal rankings
- Product preference studies (A/B/C testing)
- LLM evaluation (AlpacaEval uses Bradley-Terry for final rankings)


In [ ]:
# Cell 2: Setup and imports
import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent / "backend"))

from comparative_evaluation import BradleyTerryRanking

# Load pairwise comparisons dataset
data_path = Path("data/pairwise_comparisons.json")
with open(data_path) as f:
    comparisons_raw = json.load(f)

# Convert to Bradley-Terry format: list of (model_a, model_b, result) tuples
comparisons = []
for comp in comparisons_raw:
    model_a = "Response_A"
    model_b = "Response_B"
    # result: 1 if A wins, 0 if B wins, 0.5 if tie
    if comp['winner'] == 'A':
        result = 1
    elif comp['winner'] == 'B':
        result = 0
    else:
        result = 0.5
    
    comparisons.append({
        'model_a': model_a,
        'model_b': model_b,
        'winner': result
    })

print(f"✅ Loaded {len(comparisons)} pairwise comparisons")
print("✅ BradleyTerryRanking class imported successfully")
print("\nFirst comparison:")
print(f"Model A: {comparisons[0]['model_a']}")
print(f"Model B: {comparisons[0]['model_b']}")
print(f"Result: {comparisons[0]['winner']} (1=A wins, 0=B wins, 0.5=tie)")

## Bradley-Terry Model Explanation

### The Probabilistic Model

**Core idea:** Each model i has a latent skill parameter θᵢ

**Win probability:**
```
P(model i beats model j) = exp(θᵢ) / (exp(θᵢ) + exp(θⱼ))
                         = 1 / (1 + exp(θⱼ - θᵢ))
```

This is a **logistic function** - the same used in logistic regression!

### Maximum Likelihood Estimation (MLE)

**Likelihood function:**
```
L(θ) = ∏ P(observed comparisons | θ)
```

**Goal:** Find θ that maximizes L(θ)

**Method:** 
- Take log: `log L(θ)` (easier to optimize)
- Use iterative solver (Newton-Raphson, IRLS)
- Returns θ and standard errors

### Intuition with Examples

**Example 1: Equal skill**
- θ_A = 0.0, θ_B = 0.0
- P(A beats B) = 1 / (1 + exp(0 - 0)) = 0.5 (50%)

**Example 2: A slightly better**
- θ_A = 0.5, θ_B = 0.0
- P(A beats B) = 1 / (1 + exp(0 - 0.5)) = 0.62 (62%)

**Example 3: A much better**
- θ_A = 2.0, θ_B = 0.0
- P(A beats B) = 1 / (1 + exp(0 - 2)) = 0.88 (88%)

**Key insight:** θ difference of 0.5 ≈ 62% win rate, θ difference of 2.0 ≈ 88% win rate

### Bradley-Terry vs Elo

| Aspect | Bradley-Terry | Elo |
|--------|---------------|-----|
| **Estimation** | Batch (all data at once) | Online (sequential) |
| **Uncertainty** | Yes (standard errors) | No |
| **Order dependence** | No (order-independent) | Yes |
| **New models** | Requires refitting | Easy (assign initial rating) |
| **Complexity** | Moderate (MLE solver) | Simple (formula) |
| **Best for** | Static analysis | Dynamic leaderboard |


In [ ]:
# Cell 4: Demonstrate Bradley-Terry formula with examples
def bt_win_probability(theta_a, theta_b):
    """Calculate P(A beats B) using Bradley-Terry formula."""
    return 1 / (1 + np.exp(theta_b - theta_a))

print("=" * 60)
print("Bradley-Terry Win Probability Examples")
print("=" * 60)

examples = [
    (0.0, 0.0, "Equal skill"),
    (0.5, 0.0, "A slightly better"),
    (1.0, 0.0, "A moderately better"),
    (2.0, 0.0, "A much better"),
    (0.0, -0.5, "B slightly worse (same as A=0.5)"),
]

for theta_a, theta_b, description in examples:
    prob = bt_win_probability(theta_a, theta_b)
    print(f"\nθ_A={theta_a:4.1f}, θ_B={theta_b:5.1f} ({description})")
    print(f"  → P(A beats B) = {prob:.1%}")

# Visualize relationship
theta_diff = np.linspace(-3, 3, 100)
win_prob = 1 / (1 + np.exp(-theta_diff))

plt.figure(figsize=(10, 6))
plt.plot(theta_diff, win_prob, linewidth=3, color='steelblue')
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Even match')
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('Skill Difference (θ_A - θ_B)', fontsize=12)
plt.ylabel('P(A beats B)', fontsize=12)
plt.title('Bradley-Terry Win Probability Curve', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)

# Add annotations
plt.annotate('θ_A - θ_B = 1\n≈ 73% win rate', xy=(1, 0.73), xytext=(1.5, 0.85),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, color='red', fontweight='bold')
plt.annotate('θ_A - θ_B = -1\n≈ 27% win rate', xy=(-1, 0.27), xytext=(-2, 0.15),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=10, color='red', fontweight='bold')

plt.tight_layout()
plt.savefig('results/bradley_terry_curve.png', dpi=150, bbox_inches='tight')
print("\n\n✅ Win probability curve saved to results/bradley_terry_curve.png")
plt.show()

In [ ]:
# Cell 5: Fit Bradley-Terry model to comparison data
print("Fitting Bradley-Terry model to pairwise comparisons...\n")

# Initialize and fit model
bt = BradleyTerryRanking()
bt.fit(comparisons)

# Get rankings
rankings = bt.get_rankings()

print("=" * 60)
print("Bradley-Terry Model Results")
print("=" * 60)
print(f"Converged: {bt.converged}")
print(f"Number of iterations: {bt.n_iterations}")
print(f"Final log-likelihood: {bt.log_likelihood:.2f}")
print("\nSkill Parameters (θ):")
print("=" * 60)
print(f"{'Model':<20} {'θ (skill)':<12} {'Std Error':<12} {'95% CI'}")
print("=" * 60)

for model, skill, std_err in rankings:
    ci_lower = skill - 1.96 * std_err
    ci_upper = skill + 1.96 * std_err
    print(f"{model:<20} {skill:>8.3f}     {std_err:>8.3f}     [{ci_lower:6.3f}, {ci_upper:6.3f}]")

print("\n📊 Interpretation:")
print("- θ > 0: Better than average")
print("- θ < 0: Worse than average")
print("- θ = 0: Average skill (reference point)")
print("- Smaller std error = more confident estimate")

print("\n✅ Model fitting complete!")

In [ ]:
# Cell 6: Visualize skill estimates with uncertainty
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Skill parameters with error bars
ax1 = axes[0]
models = [m for m, _, _ in rankings]
skills = [s for _, s, _ in rankings]
std_errs = [e for _, _, e in rankings]
errors = [1.96 * e for e in std_errs]  # 95% CI

y_pos = np.arange(len(models))
colors = ['#2ecc71' if s > 0 else '#e74c3c' for s in skills]

ax1.barh(y_pos, skills, xerr=errors, color=colors, alpha=0.7,
         ecolor='black', capsize=5, error_kw={'linewidth': 2}, edgecolor='black')
ax1.set_yticks(y_pos)
ax1.set_yticklabels(models)
ax1.set_xlabel('Skill Parameter θ (with 95% CI)', fontsize=11)
ax1.set_title('Bradley-Terry Skill Estimates', fontsize=13, fontweight='bold')
ax1.axvline(x=0, color='gray', linestyle='--', alpha=0.5, linewidth=2, label='Average skill')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3, axis='x')

# Add skill values on bars
for i, (skill, error) in enumerate(zip(skills, errors)):
    x_pos = skill + error + 0.05 if skill > 0 else skill - error - 0.05
    ax1.text(x_pos, i, f'{skill:.3f}±{error:.3f}', va='center', 
             fontsize=9, fontweight='bold')

# Plot 2: Win probabilities vs average opponent
ax2 = axes[1]
# Calculate win probability vs average opponent (θ=0)
win_probs = [1 / (1 + np.exp(-s)) for s in skills]

bars = ax2.barh(y_pos, win_probs, color=colors, alpha=0.7, edgecolor='black')
ax2.set_yticks(y_pos)
ax2.set_yticklabels(models)
ax2.set_xlabel('Win Probability vs Average Opponent', fontsize=11)
ax2.set_title('Expected Win Rates', fontsize=13, fontweight='bold')
ax2.axvline(x=0.5, color='gray', linestyle='--', alpha=0.5, linewidth=2, label='50%')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3, axis='x')

# Add percentages on bars
for i, (bar, prob) in enumerate(zip(bars, win_probs)):
    ax2.text(prob + 0.02, i, f'{prob:.1%}', va='center', 
             fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('results/bradley_terry_skills.png', dpi=150, bbox_inches='tight')
print("\n✅ Skill visualization saved to results/bradley_terry_skills.png")
plt.show()

In [ ]:
# Cell 7: Compare Bradley-Terry vs Elo rankings
from comparative_evaluation import EloRanking

# Fit Elo model
elo = EloRanking(initial_rating=1500, k_factor=32)
for comp in comparisons:
    elo.record_match(comp['model_a'], comp['model_b'], comp['winner'])

elo_rankings = elo.get_leaderboard()

# Compare rankings
print("=" * 80)
print("Bradley-Terry vs Elo Rankings Comparison")
print("=" * 80)
print(f"{'Model':<20} {'BT θ':<12} {'BT Rank':<10} {'Elo Rating':<12} {'Elo Rank'}")
print("=" * 80)

# Create rank mappings
bt_ranks = {model: i+1 for i, (model, _, _) in enumerate(rankings)}
elo_ranks = {model: i+1 for i, (model, _) in enumerate(elo_rankings)}
bt_skills = {model: skill for model, skill, _ in rankings}
elo_ratings = {model: rating for model, rating in elo_rankings}

all_models = sorted(set(bt_ranks.keys()) | set(elo_ranks.keys()))

for model in all_models:
    bt_skill = bt_skills.get(model, 0.0)
    bt_rank = bt_ranks.get(model, '-')
    elo_rating = elo_ratings.get(model, 1500)
    elo_rank = elo_ranks.get(model, '-')
    
    # Highlight rank differences
    if bt_rank != '-' and elo_rank != '-':
        rank_diff = abs(bt_rank - elo_rank)
        marker = " ⚠️" if rank_diff > 0 else " ✅"
    else:
        marker = ""
    
    print(f"{model:<20} {bt_skill:>8.3f}     {str(bt_rank):<9} {elo_rating:>9.1f}     {str(elo_rank):<8}{marker}")

# Calculate rank correlation (Spearman)
from scipy.stats import spearmanr

bt_rank_list = [bt_ranks[m] for m in all_models if m in bt_ranks]
elo_rank_list = [elo_ranks[m] for m in all_models if m in elo_ranks]

if len(bt_rank_list) > 1:
    correlation, p_value = spearmanr(bt_rank_list, elo_rank_list)
    print(f"\n📊 Spearman Rank Correlation: {correlation:.3f} (p={p_value:.4f})")
    
    if correlation > 0.9:
        print("✅ Very high agreement between Bradley-Terry and Elo")
    elif correlation > 0.7:
        print("⚠️  Moderate agreement - some rank differences")
    else:
        print("❌ Low agreement - significant ranking differences")

print("\n💡 Key Differences:")
print("- Bradley-Terry: Order-independent, provides uncertainty")
print("- Elo: Order-dependent (early matches have more impact)")
print("- High correlation suggests both methods agree on relative skill")

In [ ]:
# Cell 8: Decision guide - when to use Bradley-Terry
print("=" * 70)
print("When to Use Bradley-Terry vs Elo: Decision Guide")
print("=" * 70)

decision_tree = """
START: Do you need to rank models from pairwise comparisons?
│
├─ YES → Are you building a LIVE leaderboard with continuous updates?
│        │
│        ├─ YES → Do new models get added frequently?
│        │        │
│        │        ├─ YES → Use **Elo**
│        │        │        (Easy to add new models, online updates)
│        │        │
│        │        └─ NO → Do you need uncertainty estimates?
│        │                 │
│        │                 ├─ YES → Use **Bradley-Terry** (batch refit periodically)
│        │                 │
│        │                 └─ NO → Use **Elo** (simpler)
│        │
│        └─ NO → Is this a ONE-TIME ranking of a fixed set of models?
│                 │
│                 ├─ YES → Do you need uncertainty / statistical inference?
│                 │        │
│                 │        ├─ YES → Use **Bradley-Terry**
│                 │        │        (Uncertainty, order-independent)
│                 │        │
│                 │        └─ NO → Either works, **Elo** is simpler
│                 │
│                 └─ MAYBE → Use **Elo** for exploration, **Bradley-Terry** for final report
│
└─ NO → Use different evaluation method (pointwise scoring, A/B testing, etc.)
"""

print(decision_tree)

print("\n" + "=" * 70)
print("Summary Table")
print("=" * 70)

comparison = [
    ("Criterion", "Bradley-Terry", "Elo"),
    ("-" * 30, "-" * 20, "-" * 15),
    ("Update type", "Batch (offline)", "Online (sequential)"),
    ("Uncertainty estimates", "Yes (std errors)", "No"),
    ("Order dependence", "No", "Yes"),
    ("Adding new models", "Requires refit", "Easy"),
    ("Computational cost", "Moderate (MLE)", "Low (formula)"),
    ("Statistical rigor", "High (MLE)", "Heuristic"),
    ("Best use case", "Static analysis", "Live leaderboard"),
    ("Example", "AlpacaEval final", "Chatbot Arena"),
]

for row in comparison:
    print(f"{row[0]:<30} {row[1]:<20} {row[2]:<15}")

print("\n💡 Practical Recommendation:")
print("   - Use **Elo** for exploration and live leaderboards")
print("   - Use **Bradley-Terry** for final analysis and publication")
print("   - If they disagree significantly (correlation < 0.7):")
print("     → Investigate judge inconsistency or transitivity violations")

## Summary and Key Takeaways

### What You Learned

1. **Bradley-Terry Model**:
   - Probabilistic foundation: `P(A beats B) = exp(θ_A) / (exp(θ_A) + exp(θ_B))`
   - Maximum likelihood estimation for skill parameters
   - Provides uncertainty via standard errors

2. **Batch vs Online**:
   - Bradley-Terry: Uses all data simultaneously (order-independent)
   - Elo: Sequential updates (order-dependent)
   - Both converge to similar rankings with enough data

3. **Uncertainty Quantification**:
   - Standard errors from MLE
   - 95% confidence intervals for skill estimates
   - Enables statistical hypothesis testing

4. **When to Use Each**:
   - Bradley-Terry: Static analysis, need uncertainty, publication
   - Elo: Live leaderboard, frequent model additions, simplicity

### Practical Guidelines

**Bradley-Terry strengths:**
- ✅ Order-independent (fair to all models)
- ✅ Provides uncertainty (can test "is A significantly better than B?")
- ✅ Statistically principled (MLE)
- ✅ Works with incomplete comparison data (not all pairs compared)

**Bradley-Terry limitations:**
- ❌ Requires batch refitting for new models
- ❌ More complex to implement
- ❌ Assumes transitivity (fit degrades with many violations)
- ❌ Can be slow for large datasets (O(n²) comparisons)

**Elo strengths:**
- ✅ Simple and fast
- ✅ Easy to add new models
- ✅ Well-understood (70+ years of use)
- ✅ Robust to transitivity violations

**Elo limitations:**
- ❌ Order-dependent (early matches have outsized impact)
- ❌ No uncertainty estimates
- ❌ Requires K-factor tuning
- ❌ Cold start problem for new models

### Real-World Applications

**Bradley-Terry:**
- AlpacaEval 2.0 (final win rates)
- MT-Bench (model comparison)
- Academic research (statistical testing)

**Elo:**
- Chatbot Arena (live leaderboard)
- Game matchmaking (LoL, Overwatch)
- Chess ratings (FIDE)

### Next Steps

- 🧪 [A/B Testing vs Comparative Eval](ab_testing_vs_comparative_eval.ipynb) - Sample size comparison
- 📖 [Comparative Evaluation Guide](comparative_evaluation_guide.md) - Full methodology
- 🎯 [Elo Ranking Tutorial](elo_ranking_tutorial.ipynb) - Dynamic leaderboards

### Resources

- [Bradley-Terry Model (Wikipedia)](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model)
- [AlpacaEval 2.0 Paper](https://arxiv.org/abs/2404.04475) - Uses Bradley-Terry
- [Backend Implementation](../backend/comparative_evaluation.py) - BradleyTerryRanking source

---

**🎉 Congratulations!** You now understand probabilistic ranking with Bradley-Terry and can choose the right algorithm for your use case.
